In [1]:
# Version 2 of EEG imagined speech project
# Used https://www.kaggle.com/code/widhiwinata/grasp-and-lift-movement-detection-with-dl
# Uses each line of data as input. Results are not above chance (About 6%)
# Uses EEG dataset: https://zenodo.org/record/3554128#.Ye85WC-cby8 
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from glob import glob
import scipy
from scipy.signal import butter
from scipy.signal import freqz
from scipy.fftpack import fft, ifft
import os

from sklearn.preprocessing import StandardScaler

In [2]:
def wavelet_denoising(x, wavelet='db2', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [3]:
def prepare_data_train(file_path):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(file_path)
    # events file
    events_file_path = file_path.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_file_path)
    clean=data.drop(['Time:256Hz' ], axis=1)#remove id
    labels=labels.drop(['Time:256Hz' ], axis=1)#remove id
    return  clean,labels

In [4]:
def prepare_data_test(file_path):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(file_path)
    # events file
    events_file_path = file_path.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_file_path)
    clean=data.drop(['Time:256Hz' ], axis=1)#remove id
    labels=labels.drop(['Time:256Hz' ], axis=1)#remove id
    return  clean,labels

def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    return X_prep

def data_preprocess_test(X):
    X_prep=scaler.transform(X)
    return X_prep

subjects = range(1,22)
ids_tot = []
pred_tot = []
X_train_butter = []

In [5]:
y_raw= []
raw = []
y_rawt= []
rawt = []

In [6]:
for subject in subjects:
    
#Read Data
    file_paths =  sorted(glob('fixed_EEG/training_csvfiles/sub%d_data_train.csv' % (subject)))
    print(file_paths)

    for file_path in file_paths:
      data,labels=prepare_data_train(file_path)
      raw.append(data)
      y_raw.append(labels)

['fixed_EEG/training_csvfiles/sub1_data_train.csv']
['fixed_EEG/training_csvfiles/sub2_data_train.csv']
['fixed_EEG/training_csvfiles/sub3_data_train.csv']
['fixed_EEG/training_csvfiles/sub4_data_train.csv']
['fixed_EEG/training_csvfiles/sub5_data_train.csv']
['fixed_EEG/training_csvfiles/sub6_data_train.csv']
['fixed_EEG/training_csvfiles/sub7_data_train.csv']
['fixed_EEG/training_csvfiles/sub8_data_train.csv']
['fixed_EEG/training_csvfiles/sub9_data_train.csv']
['fixed_EEG/training_csvfiles/sub10_data_train.csv']
['fixed_EEG/training_csvfiles/sub11_data_train.csv']
['fixed_EEG/training_csvfiles/sub12_data_train.csv']
['fixed_EEG/training_csvfiles/sub13_data_train.csv']
['fixed_EEG/training_csvfiles/sub14_data_train.csv']
['fixed_EEG/training_csvfiles/sub15_data_train.csv']
['fixed_EEG/training_csvfiles/sub16_data_train.csv']
['fixed_EEG/training_csvfiles/sub17_data_train.csv']
['fixed_EEG/training_csvfiles/sub18_data_train.csv']
['fixed_EEG/training_csvfiles/sub19_data_train.csv']
['

## Wavelet Denoising

In [7]:
X = pd.concat(raw)
y = pd.concat(y_raw)
#transform in numpy array
#transform train data into numpy array
X_train =np.asarray(X.astype(float))
y_train = np.asarray(y.astype(float))

from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler
scaler= StandardScaler()
def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    return X_prep

x_train_butter=wavelet_denoising(X_train)
x_train=data_preprocess_train(x_train_butter)
splitrate=-x_train.shape[0]//5*2
xval=x_train[splitrate:splitrate//2]
yval=y_train[splitrate:splitrate//2]
xtest=x_train[splitrate//2:]
ytest=y_train[splitrate//2:]
xtrain=x_train[:splitrate]
ytrain=y_train[:splitrate]

In [8]:
print(len(raw))
print(len(y_raw))
print(ytrain.shape)
print(xtrain.shape)

21
21
(2035200, 16)
(2035200, 14)


In [ ]:
def wavelet_denoising(x, wavelet='db2', level=3):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

signal=pd.read_csv('fixed_EEG/training_csvfiles/sub1_data_train.csv')
signal = signal.drop("Time:256Hz", axis=1)
filtered = wavelet_denoising(signal, wavelet='db2', level=3)

fig, ax = plt.subplots(figsize=(8,5))
ax.plot(signal.iloc[:10000,1], label='signal', color="b", alpha=0.5,)
ax.plot(filtered[:10000,1], label='reconstructed signal',color="k")
ax.legend(loc='upper left')
ax.set_title('Denoising with DWT')
plt.show()

In [ ]:
import seaborn as sns
ye=pd.DataFrame(y_train)

ye.columns=["fleece ","goose ","trap ","thought ","m ","n ","ng ","f ", "s ", "sh ", "v ", "z ", 
            "zh ", "p ", "t ", "k "]
categories = list(ye.columns.values)
sns.set(font_scale = 1)
plt.figure(figsize=(15,8))
ax= sns.barplot(categories, ye.iloc[:,0:].sum().values)
plt.title("Number of samples labeled as active (1) out of {0} length data".format((ye.shape[0])),fontsize=20)

plt.ylabel('Number of events', fontsize=18)
plt.xlabel('Event Type ', fontsize=12)
#adding the text labels
rects = ax.patches
labels = ye.iloc[:,0:].sum().values

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=10)
plt.show()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
import tensorflow as tf
!module load gpu

In [ ]:
!module load gpu
load = 1
time_steps = 1000
subsample = 50
model = Sequential()
model.add(Conv2D(filters = 28, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 28, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 28, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(14, activation = "relu"))
model.add(BatchNormalization())

model.add(Dense(16, activation = "sigmoid"))


adam = Adam(lr = 0.0001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ['accuracy','mse'])

model.summary()



In [ ]:
def valgenerator():
    while 1:
        batch_size=14
        x_time_data = np.zeros((batch_size, time_steps//subsample, 14))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xval)-time_steps)
            x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
            yy.append(yval[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy
    

In [ ]:
import time
start=time.time()
def generator(batch_size):
    while 1:
        
        x_time_data = np.zeros((batch_size, time_steps//subsample, 14))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtrain)-time_steps)
            x_time_data[i] = xtrain[random_index:random_index+time_steps:subsample]
            yy.append(ytrain[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1],  x_time_data.shape[2],1)), yy
#originally 600 steps_per_epoch, 50 epochs        
history =model.fit_generator(generator(14), steps_per_epoch = 900, epochs = 100,validation_data=valgenerator(),
                              validation_steps=800)

print('training time taken: ',round(time.time()-start,0),'seconds')



In [ ]:
def plot_training_history(history):
    acc = history.history['accuracy']
    loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
 
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
    plt.plot(epochs, val_acc, 'r--', label='Validation acc',linewidth=2)
    plt.title('Training  accuracy')
    plt.legend()
 
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'k', label='Validationloss ')
    plt.title('Training loss')
    plt.legend()
 
    plt.show()
plot_training_history(history)

In [ ]:
time_steps = 1000
subsample = 50
def val_generator():
    while 1:
        batch_size = 1
        x_time_data = np.zeros((batch_size, time_steps//subsample, 14))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtest)-time_steps)
            x_time_data[i] = xtest[random_index:random_index+time_steps:subsample]
            yy.append(ytest[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [ ]:
gen_data = val_generator()
scores = []
num_test =5000
for i in range(num_test):
    x_test, y_test = next(gen_data)
    #print(y_test)
    while not 1 in y_test:
        #print(x_test)
        x_test, y_test = next(gen_data)
    score=model.evaluate(x_test, y_test, verbose=0)

    scores.append(score[1])
scores = np.asarray(scores)

print("Accuracy ", np.mean(scores))
print(scores)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score
lr = SGDClassifier()
#clf = OneVsRestClassifier(LinearSVC(random_state=0))
clf =OneVsRestClassifier(lr)
import time
start=time.time()
clf.fit(xtrain,ytrain)

print('training time taken: ',round(time.time()-start,0),'seconds')

In [ ]:
from sklearn.metrics import roc_curve, auc
y_score = clf.decision_function(xval)
#y_score = clf.predict(xval)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(6):
    fpr[i], tpr[i], _ = roc_curve(yval[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(6):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import label_ranking_average_precision_score
predictions = clf.predict(xval)
# accuracy
print("roc_auc:",sum(roc_auc.values())/6)
print("Accuracy = ",accuracy_score(yval,predictions))
print("label_ranking_average_precision_score",label_ranking_average_precision_score(yval,predictions))